In [10]:
import pandas as pd
import numpy as np
import keras
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import hashlib
import math, os
from tqdm import tqdm

In [11]:
pd.set_option('display.max_columns', 40)

In [12]:
# トレーニング用の入力データの選択
#X_columns = ['locality', 'age', 'rank', 'leg', 'racing piont', \
#             'S', 'B', 'Nige', 'Maki', 'Sashi', 'Ma', \
#             '1st', '2nd', '3rd', 'Chakugai', 'win', '2ren', '3ren']
X_columns = ['1st', '2nd', '3rd', 'Chakugai', 'win', '2ren', '3ren']


# Define Model

In [13]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import GRU
from keras.optimizers import Adam
from keras.layers.normalization import BatchNormalization
from keras.models import load_model

In [14]:
n_in = len(X_columns) * 9
n_out = 9
n_hidden = 1024

In [15]:
model = Sequential()
model.add(Dense(n_hidden, input_shape=(n_in, )))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(n_hidden))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(n_out))
model.add(BatchNormalization())
model.add(Activation('softmax'))

In [16]:
optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [17]:
filename = 'model/DNN_model.h5'
model.save(filename)

# 関数、クラス　定義

In [18]:
def get_train_test_data(df_train):
    X = []
    target = []
        
    # 各レース毎に
    grouped = df_train.groupby(['date', 'place', 'race_num'])
    for race_name, group in tqdm(grouped):
        #print(race_name)
        racer_count = group.shape[0]
        # もし、９輪ではないレースは、トレーニングの対象から外す（モデルを固めるため）
        if racer_count != 9:
            continue
        X.append(group[X_columns].values)
        target.append(group['target'].values)

    X = np.array(X, dtype='float')
    X = X.reshape(X.shape[0], X.shape[1] * X.shape[2])
    d_ = np.array(target)

    X_train, X_test, d_train, d_test = train_test_split(X, d_, test_size = 0.2)

    return X_train, X_test, d_train, d_test


# 学習

### 学習データを読み込む

In [19]:
df_train = pd.read_csv('df_train.csv')

c:\users\ttsuchiy\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (4,10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [20]:
print("Generating Training/Test Data")
X_train, X_test, Y_train, Y_test = get_train_test_data(df_train)

Generating Training/Test Data


100%|█████████████████████████████████| 146536/146536 [04:33<00:00, 535.59it/s]


In [25]:
Y_train

array([[0.11111111, 0.25      , 0.13888889, ..., 0.16666667, 0.02777778,
        0.19444444],
       [0.25      , 0.11111111, 0.13888889, ..., 0.22222222, 0.16666667,
        0.19444444],
       [0.22222222, 0.13888889, 0.16666667, ..., 0.25      , 0.02777778,
        0.19444444],
       ...,
       [0.19444444, 0.11111111, 0.05555556, ..., 0.22222222, 0.08333333,
        0.13888889],
       [0.25      , 0.19444444, 0.16666667, ..., 0.22222222, 0.13888889,
        0.11111111],
       [0.13888889, 0.16666667, 0.05555556, ..., 0.02777778, 0.11111111,
        0.22222222]])

# Training

In [23]:
batch_size = 32
epochs = 120

In [24]:
print("Training ...")
model.fit(X_train, Y_train, epochs = epochs, batch_size=batch_size)

loss_and_metrics = model.evaluate(X_test, Y_test)
print(loss_and_metrics)

model.save(filename)


Training ...
Epoch 1/1


10016/75225 [==>...........................] - ETA: 2:55 - loss: 3.329 - ETA: 2:33 - loss: 3.259 - ETA: 2:33 - loss: 3.204 - ETA: 2:31 - loss: 3.142 - ETA: 2:31 - loss: 3.102 - ETA: 2:27 - loss: 3.062 - ETA: 2:29 - loss: 3.023 - ETA: 2:27 - loss: 2.994 - ETA: 2:27 - loss: 2.967 - ETA: 2:26 - loss: 2.946 - ETA: 2:26 - loss: 2.926 - ETA: 2:27 - loss: 2.912 - ETA: 2:27 - loss: 2.898 - ETA: 2:27 - loss: 2.886 - ETA: 2:29 - loss: 2.876 - ETA: 2:31 - loss: 2.867 - ETA: 2:34 - loss: 2.859 - ETA: 2:35 - loss: 2.852 - ETA: 2:37 - loss: 2.845 - ETA: 2:38 - loss: 2.839 - ETA: 2:40 - loss: 2.834 - ETA: 2:40 - loss: 2.829 - ETA: 2:40 - loss: 2.825 - ETA: 2:40 - loss: 2.821 - ETA: 2:40 - loss: 2.817 - ETA: 2:40 - loss: 2.814 - ETA: 2:40 - loss: 2.810 - ETA: 2:40 - loss: 2.807 - ETA: 2:40 - loss: 2.804 - ETA: 2:42 - loss: 2.802 - ETA: 2:41 - loss: 2.799 - ETA: 2:41 - loss: 2.797 - ETA: 2:42 - loss: 2.795 - ETA: 2:42 - loss: 2.793 - ETA: 2:42 - loss: 2.791 - ETA: 2:42 - loss: 2.789 - ETA: 2:42 - loss:

20064/75225 [=======>......................] - ETA: 2:32 - loss: 2.731 - ETA: 2:32 - loss: 2.731 - ETA: 2:32 - loss: 2.731 - ETA: 2:32 - loss: 2.731 - ETA: 2:32 - loss: 2.731 - ETA: 2:32 - loss: 2.731 - ETA: 2:32 - loss: 2.731 - ETA: 2:32 - loss: 2.730 - ETA: 2:32 - loss: 2.730 - ETA: 2:32 - loss: 2.730 - ETA: 2:32 - loss: 2.730 - ETA: 2:31 - loss: 2.730 - ETA: 2:31 - loss: 2.730 - ETA: 2:31 - loss: 2.730 - ETA: 2:31 - loss: 2.730 - ETA: 2:31 - loss: 2.730 - ETA: 2:31 - loss: 2.730 - ETA: 2:31 - loss: 2.730 - ETA: 2:31 - loss: 2.730 - ETA: 2:31 - loss: 2.730 - ETA: 2:31 - loss: 2.730 - ETA: 2:31 - loss: 2.730 - ETA: 2:31 - loss: 2.730 - ETA: 2:30 - loss: 2.730 - ETA: 2:30 - loss: 2.730 - ETA: 2:30 - loss: 2.730 - ETA: 2:30 - loss: 2.730 - ETA: 2:30 - loss: 2.730 - ETA: 2:30 - loss: 2.730 - ETA: 2:30 - loss: 2.730 - ETA: 2:30 - loss: 2.730 - ETA: 2:30 - loss: 2.730 - ETA: 2:30 - loss: 2.730 - ETA: 2:29 - loss: 2.730 - ETA: 2:29 - loss: 2.730 - ETA: 2:29 - loss: 2.730 - ETA: 2:29 - loss:

30112/75225 [===========>..................] - ETA: 2:06 - loss: 2.726 - ETA: 2:06 - loss: 2.726 - ETA: 2:06 - loss: 2.726 - ETA: 2:05 - loss: 2.726 - ETA: 2:05 - loss: 2.726 - ETA: 2:05 - loss: 2.726 - ETA: 2:05 - loss: 2.726 - ETA: 2:05 - loss: 2.726 - ETA: 2:05 - loss: 2.726 - ETA: 2:05 - loss: 2.726 - ETA: 2:05 - loss: 2.726 - ETA: 2:05 - loss: 2.726 - ETA: 2:05 - loss: 2.726 - ETA: 2:05 - loss: 2.726 - ETA: 2:05 - loss: 2.726 - ETA: 2:04 - loss: 2.726 - ETA: 2:04 - loss: 2.726 - ETA: 2:04 - loss: 2.726 - ETA: 2:04 - loss: 2.726 - ETA: 2:04 - loss: 2.726 - ETA: 2:04 - loss: 2.726 - ETA: 2:04 - loss: 2.726 - ETA: 2:04 - loss: 2.726 - ETA: 2:04 - loss: 2.726 - ETA: 2:04 - loss: 2.726 - ETA: 2:04 - loss: 2.726 - ETA: 2:03 - loss: 2.726 - ETA: 2:03 - loss: 2.726 - ETA: 2:03 - loss: 2.726 - ETA: 2:03 - loss: 2.726 - ETA: 2:03 - loss: 2.726 - ETA: 2:03 - loss: 2.726 - ETA: 2:03 - loss: 2.726 - ETA: 2:03 - loss: 2.726 - ETA: 2:03 - loss: 2.726 - ETA: 2:03 - loss: 2.726 - ETA: 2:03 - loss:

40160/75225 [===============>..............] - ETA: 1:39 - loss: 2.725 - ETA: 1:39 - loss: 2.725 - ETA: 1:39 - loss: 2.725 - ETA: 1:39 - loss: 2.725 - ETA: 1:39 - loss: 2.725 - ETA: 1:39 - loss: 2.725 - ETA: 1:39 - loss: 2.725 - ETA: 1:39 - loss: 2.725 - ETA: 1:39 - loss: 2.725 - ETA: 1:39 - loss: 2.725 - ETA: 1:39 - loss: 2.725 - ETA: 1:39 - loss: 2.725 - ETA: 1:38 - loss: 2.725 - ETA: 1:38 - loss: 2.725 - ETA: 1:38 - loss: 2.725 - ETA: 1:38 - loss: 2.725 - ETA: 1:38 - loss: 2.725 - ETA: 1:38 - loss: 2.725 - ETA: 1:38 - loss: 2.725 - ETA: 1:38 - loss: 2.725 - ETA: 1:38 - loss: 2.725 - ETA: 1:38 - loss: 2.725 - ETA: 1:38 - loss: 2.725 - ETA: 1:38 - loss: 2.725 - ETA: 1:38 - loss: 2.725 - ETA: 1:37 - loss: 2.725 - ETA: 1:37 - loss: 2.725 - ETA: 1:37 - loss: 2.725 - ETA: 1:37 - loss: 2.725 - ETA: 1:37 - loss: 2.725 - ETA: 1:37 - loss: 2.725 - ETA: 1:37 - loss: 2.725 - ETA: 1:37 - loss: 2.725 - ETA: 1:37 - loss: 2.725 - ETA: 1:37 - loss: 2.725 - ETA: 1:37 - loss: 2.725 - ETA: 1:37 - loss:

50368/75225 [===================>..........] - ETA: 1:17 - loss: 2.724 - ETA: 1:17 - loss: 2.724 - ETA: 1:17 - loss: 2.724 - ETA: 1:17 - loss: 2.724 - ETA: 1:17 - loss: 2.724 - ETA: 1:17 - loss: 2.724 - ETA: 1:16 - loss: 2.724 - ETA: 1:16 - loss: 2.724 - ETA: 1:16 - loss: 2.724 - ETA: 1:16 - loss: 2.724 - ETA: 1:16 - loss: 2.724 - ETA: 1:16 - loss: 2.724 - ETA: 1:16 - loss: 2.724 - ETA: 1:16 - loss: 2.724 - ETA: 1:16 - loss: 2.724 - ETA: 1:16 - loss: 2.724 - ETA: 1:16 - loss: 2.724 - ETA: 1:16 - loss: 2.724 - ETA: 1:16 - loss: 2.724 - ETA: 1:16 - loss: 2.724 - ETA: 1:16 - loss: 2.724 - ETA: 1:15 - loss: 2.724 - ETA: 1:15 - loss: 2.724 - ETA: 1:15 - loss: 2.724 - ETA: 1:15 - loss: 2.724 - ETA: 1:15 - loss: 2.724 - ETA: 1:15 - loss: 2.724 - ETA: 1:15 - loss: 2.724 - ETA: 1:15 - loss: 2.724 - ETA: 1:15 - loss: 2.724 - ETA: 1:15 - loss: 2.724 - ETA: 1:15 - loss: 2.724 - ETA: 1:15 - loss: 2.724 - ETA: 1:15 - loss: 2.724 - ETA: 1:15 - loss: 2.724 - ETA: 1:14 - loss: 2.724 - ETA: 1:14 - loss:

61248/75225 [=======================>......] - ETA: 54s - loss: 2.72 - ETA: 54s - loss: 2.72 - ETA: 54s - loss: 2.72 - ETA: 54s - loss: 2.72 - ETA: 54s - loss: 2.72 - ETA: 54s - loss: 2.72 - ETA: 54s - loss: 2.72 - ETA: 54s - loss: 2.72 - ETA: 54s - loss: 2.72 - ETA: 54s - loss: 2.72 - ETA: 54s - loss: 2.72 - ETA: 53s - loss: 2.72 - ETA: 53s - loss: 2.72 - ETA: 53s - loss: 2.72 - ETA: 53s - loss: 2.72 - ETA: 53s - loss: 2.72 - ETA: 53s - loss: 2.72 - ETA: 53s - loss: 2.72 - ETA: 53s - loss: 2.72 - ETA: 53s - loss: 2.72 - ETA: 53s - loss: 2.72 - ETA: 53s - loss: 2.72 - ETA: 53s - loss: 2.72 - ETA: 53s - loss: 2.72 - ETA: 53s - loss: 2.72 - ETA: 52s - loss: 2.72 - ETA: 52s - loss: 2.72 - ETA: 52s - loss: 2.72 - ETA: 52s - loss: 2.72 - ETA: 52s - loss: 2.72 - ETA: 52s - loss: 2.72 - ETA: 52s - loss: 2.72 - ETA: 52s - loss: 2.72 - ETA: 52s - loss: 2.72 - ETA: 52s - loss: 2.72 - ETA: 52s - loss: 2.72 - ETA: 52s - loss: 2.72 - ETA: 52s - loss: 2.72 - ETA: 51s - loss: 2.72 - ETA: 51s - loss: 

72128/75225 [===========================>..] - ETA: 30s - loss: 2.72 - ETA: 30s - loss: 2.72 - ETA: 30s - loss: 2.72 - ETA: 30s - loss: 2.72 - ETA: 30s - loss: 2.72 - ETA: 30s - loss: 2.72 - ETA: 30s - loss: 2.72 - ETA: 30s - loss: 2.72 - ETA: 30s - loss: 2.72 - ETA: 29s - loss: 2.72 - ETA: 29s - loss: 2.72 - ETA: 29s - loss: 2.72 - ETA: 29s - loss: 2.72 - ETA: 29s - loss: 2.72 - ETA: 29s - loss: 2.72 - ETA: 29s - loss: 2.72 - ETA: 29s - loss: 2.72 - ETA: 29s - loss: 2.72 - ETA: 29s - loss: 2.72 - ETA: 29s - loss: 2.72 - ETA: 29s - loss: 2.72 - ETA: 29s - loss: 2.72 - ETA: 29s - loss: 2.72 - ETA: 28s - loss: 2.72 - ETA: 28s - loss: 2.72 - ETA: 28s - loss: 2.72 - ETA: 28s - loss: 2.72 - ETA: 28s - loss: 2.72 - ETA: 28s - loss: 2.72 - ETA: 28s - loss: 2.72 - ETA: 28s - loss: 2.72 - ETA: 28s - loss: 2.72 - ETA: 28s - loss: 2.72 - ETA: 28s - loss: 2.72 - ETA: 28s - loss: 2.72 - ETA: 28s - loss: 2.72 - ETA: 28s - loss: 2.72 - ETA: 27s - loss: 2.72 - ETA: 27s - loss: 2.72 - ETA: 27s - loss: 

18807/18807 [==============================] - ETA: 4: - ETA: 40s - ETA: 31 - ETA: 21 - ETA: 18 - ETA: 17 - ETA: 15 - ETA: 14 - ETA: 13 - ETA: 12 - ETA: 12 - ETA: 11 - ETA: 11 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 9 - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 7s 354us/step
2